# RankNet

In [6]:
import torch

In [1]:
%%latex
$$C_{ij}=C(o_{ij})=-\bar{P_{ij}}log(P_{ij})-(1-\bar{P_{ij}})log(1-P_{ij})$$

<IPython.core.display.Latex object>

In [2]:
%%latex
$$o_{ij}=f(x_i)-f(x_j)$$

<IPython.core.display.Latex object>

In [3]:
%%latex
$$P_{ij}=\frac{e^{o_{ij}}}{1+e^{o_{ij}}}$$

<IPython.core.display.Latex object>

In [ ]:
torch.sigmoid()

In [ ]:
%%latex
$$\text{out}_{i} = \frac{1}{1 + e^{-\text{input}_{i}}}$$

In [7]:
class RankNet(torch.nn.Module):
    def __init__(self, num_input_features, hidden_dim=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1),
        )
        
        self.out_activation = torch.nn.Sigmoid()

    def forward(self, input_1, input_2):
        logits_1 = self.predict(input_1)
        logits_2 = self.predict(input_2)
        
        logits_diff = logits_1 - logits_2
        out = self.out_activation(logits_diff)

        return out
    
    def predict(self, inp):
        logits = self.model(inp)
        return logits

In [8]:
ranknet_model = RankNet(num_input_features=10)

In [12]:
inp_1, inp_2 = torch.rand(4, 10), torch.rand(4, 10)
# batch_size x input_dim
inp_2

tensor([[0.1255, 0.7267, 0.6673, 0.8020, 0.8569, 0.9031, 0.3431, 0.8151, 0.2969,
         0.5085],
        [0.2491, 0.1846, 0.5508, 0.2171, 0.0508, 0.9934, 0.4131, 0.0370, 0.2644,
         0.2890],
        [0.2778, 0.5740, 0.0688, 0.3176, 0.1460, 0.9584, 0.0766, 0.0802, 0.4292,
         0.7376],
        [0.6586, 0.5865, 0.0144, 0.9973, 0.9092, 0.0504, 0.1252, 0.0478, 0.6712,
         0.3977]])

In [20]:
preds = ranknet_model(inp_1, inp_2)
preds

tensor([[0.5054],
        [0.4507],
        [0.4879],
        [0.5165]], grad_fn=<SigmoidBackward>)

In [14]:
first_linear_layer = ranknet_model.model[0]

In [15]:
first_linear_layer.weight.grad

In [21]:
criterion = torch.nn.BCELoss()
loss = criterion(preds, torch.ones_like(preds))
loss.backward()

In [22]:
first_linear_layer.weight.grad

tensor([[ 0.0210,  0.0063,  0.0176,  0.0121,  0.0129, -0.0215,  0.0111,  0.0164,
          0.0076,  0.0059],
        [-0.0024, -0.0007, -0.0020, -0.0014, -0.0015,  0.0025, -0.0013, -0.0019,
         -0.0009, -0.0007],
        [ 0.0184,  0.0056,  0.0154,  0.0106,  0.0113, -0.0189,  0.0097,  0.0144,
          0.0066,  0.0051],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [ 0.0276,  0.0346,  0.0025,  0.0386,  0.0308,  0.0308,  0.0060,  0.0038,
          0.0326,  0.0341],
        [ 0.0106,  0.0042,  0.0053,  0.0043,  0.0043, -0.0092, -0.0008, -0.0068,
          0.0107,  0.0044],
        [ 0.0494,  0.0149,  0.0415,  0.0285,  0.0304, -0.0507,  0.0262,  0.0387,
          0.0178,  0.0138],
        [-0.0062,  0.0157,  0.0125,  0.0027, -0.0008,  0.0662, -0.0011, -0.0130,
          0.0010,  0.0172],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000],
        [-0.0321, -

In [18]:
ranknet_model.zero_grad()

# ListNet

In [23]:
from itertools import combinations
import numpy as np

from utils import ndcg, num_swapped_pairs

In [24]:
class ListNet(torch.nn.Module):
    def __init__(self, num_input_features, hidden_dim=10):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.model = torch.nn.Sequential(
            torch.nn.Linear(num_input_features, self.hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(self.hidden_dim, 1),
        )


    def forward(self, input_1):
        logits = self.model(input_1)
        return logits


In [4]:
%%latex
$$CE = -\sum ^{N}_{j=1} (P_y^i(j) * log(P_z^i(j)))$$

<IPython.core.display.Latex object>

In [5]:
%%latex
$$\text{Softmax}(x_{i}) = \frac{\exp(x_i)}{\sum_j \exp(x_j)}$$

<IPython.core.display.Latex object>

In [32]:
def listnet_ce_loss(y_i, z_i):
    """
    y_i: (n_i, 1) GT
    z_i: (n_i, 1) preds
    """

    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i))

def listnet_kl_loss(y_i, z_i):
    """
    y_i: (n_i, 1) GT
    z_i: (n_i, 1) preds
    """
    P_y_i = torch.softmax(y_i, dim=0)
    P_z_i = torch.softmax(z_i, dim=0)
    return -torch.sum(P_y_i * torch.log(P_z_i/P_y_i))


def make_dataset(N_train, N_valid, vector_dim):
    fake_weights = torch.randn(vector_dim, 1)

    X_train = torch.randn(N_train, vector_dim)
    X_valid = torch.randn(N_valid, vector_dim)

    ys_train_score = torch.mm(X_train, fake_weights)
    ys_train_score += torch.randn_like(ys_train_score)

    ys_valid_score = torch.mm(X_valid, fake_weights)
    ys_valid_score += torch.randn_like(ys_valid_score)

#     bins = [-1, 1]  # 3 relevances
    bins = [-1, 0, 1, 2]  # 5 relevances
    ys_train_rel = torch.Tensor(
        np.digitize(ys_train_score.clone().detach().numpy(), bins=bins)
    )
    ys_valid_rel = torch.Tensor(
        np.digitize(ys_valid_score.clone().detach().numpy(), bins=bins)
    )

    return X_train, X_valid, ys_train_rel, ys_valid_rel

In [38]:
N_train = 1000
N_valid = 500

vector_dim = 100
epochs = 2

batch_size = 16

X_train, X_valid, ys_train, ys_valid = make_dataset(N_train, N_valid, vector_dim)

net = ListNet(num_input_features=vector_dim)
opt = torch.optim.Adam(net.parameters())


In [34]:
torch.unique(ys_train)

tensor([0., 1., 2., 3., 4.])

In [ ]:
for epoch in range(epochs):
    idx = torch.randperm(N_train)

    X_train = X_train[idx]
    ys_train = ys_train[idx]

    cur_batch = 0
    for it in range(N_train // batch_size):
        batch_X = X_train[cur_batch: cur_batch + batch_size]
        batch_ys = ys_train[cur_batch: cur_batch + batch_size]
        cur_batch += batch_size

        opt.zero_grad()
        if len(batch_X) > 0:
            batch_pred = net(batch_X)
            batch_loss = listnet_kl_loss(batch_ys, batch_pred)
#             batch_loss = listnet_ce_loss(batch_ys, batch_pred)
            batch_loss.backward(retain_graph=True)
            opt.step()

        if it % 10 == 0:
            with torch.no_grad():
                valid_pred = net(X_valid)
                valid_swapped_pairs = num_swapped_pairs(ys_valid, valid_pred)
                ndcg_score = ndcg(ys_valid, valid_pred)
            print(f"epoch: {epoch + 1}.\tNumber of swapped pairs: " 
                  f"{valid_swapped_pairs}/{N_valid * (N_valid - 1) // 2}\t"
                  f"nDCG: {ndcg_score:.4f}")

epoch: 1.	Number of swapped pairs: 35372/124750	nDCG: 0.8203
epoch: 1.	Number of swapped pairs: 32028/124750	nDCG: 0.8374
epoch: 1.	Number of swapped pairs: 29056/124750	nDCG: 0.8508
epoch: 1.	Number of swapped pairs: 26233/124750	nDCG: 0.8656
epoch: 1.	Number of swapped pairs: 23534/124750	nDCG: 0.8791
epoch: 1.	Number of swapped pairs: 21211/124750	nDCG: 0.8901
epoch: 1.	Number of swapped pairs: 18921/124750	nDCG: 0.9020
epoch: 2.	Number of swapped pairs: 18486/124750	nDCG: 0.9056
epoch: 2.	Number of swapped pairs: 16303/124750	nDCG: 0.9183
epoch: 2.	Number of swapped pairs: 14599/124750	nDCG: 0.9396
epoch: 2.	Number of swapped pairs: 13389/124750	nDCG: 0.9491
epoch: 2.	Number of swapped pairs: 12266/124750	nDCG: 0.9592
epoch: 2.	Number of swapped pairs: 11215/124750	nDCG: 0.9675
